# Tensor Ring Convolutional Neural Network

[Open](https://colab.research.google.com/github/Bihaqo/t3f/blob/develop/docs/tutorials/riemannian.ipynb) **this page in an interactive mode via Google Colaboratory.**

Riemannian optimization is a framework for solving optimization problems with a constraint that the solution belongs to a manifold. 

Let us consider the following problem. Given some TT tensor $A$ with large tt-ranks we would like to find a tensor $X$ (with small prescribed tt-ranks $r$) which is closest to $A$ (in the sense of Frobenius norm). Mathematically it can be written as follows:
\begin{equation*}
\begin{aligned}
& \underset{X}{\text{minimize}} 
& & \frac{1}{2}\|X - A\|_F^2 \\
& \text{subject to} 
& & \text{tt_rank}(X) = r
\end{aligned}
\end{equation*}

It is known that the set of TT tensors with elementwise fixed TT ranks forms a manifold. Thus we can solve this problem using the so called Riemannian gradient descent. Given some functional $F$ on a manifold $\mathcal{M}$  it is defined as
$$\hat{x}_{k+1} = x_{k} - \alpha P_{T_{x_k}\mathcal{M}} \nabla F(x_k),$$
$$x_{k+1} = \mathcal{R}(\hat{x}_{k+1})$$
with $P_{T_{x_k}} \mathcal{M}$ being the projection onto the tangent space of $\mathcal{M}$ at the point $x_k$ and $\mathcal{R}$ being a retraction - an operation which projects points to the manifold, and $\alpha$ is the learning rate.

We can implement this in `t3f` using the `t3f.riemannian` module. As a retraction it is convenient to use the rounding method (`t3f.round`).

In [1]:
# Import numpy
import numpy as np

np.random.seed(0)

TensorFlow 2.x selected.


In [0]:
# Initialize A randomly, with large tt-ranks
shape = 10 * [2]

It is intructive to compare the obtained result with the quasioptimum delivered by the TT-round procedure.

We see that the value is slightly bigger than the exact minimum, but TT-round is faster and cheaper to compute, so it is often used in practice.
